In [ ]:
import myterial
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from neuroconnect.matrix import (
    convert_mouse_data,
    load_matrix_data,
    print_args_dict,
    matrix_vis,
    mpf_probe_connectome,
)
from neuroconnect.atlas import (
    place_probes_at_com,
    get_brain_region_meshes,
    get_n_random_points_in_region,
    get_idx_of_points_in_meshes,
)
from neuroconnect.connect_math import (
    get_dist_mean,
    get_dist_var,
    discretised_rv,
    new_dist_upper_bound,
)
from scipy.stats import truncexpon


In [ ]:
A_name = "MOp"
B_name = "SSp-ll"

vis_full = False
hemisphere = "right"
block_size_sub = 10

In [ ]:
# Load in mouse data
convert_mouse_data(A_name, B_name)
to_use = [True] * 4
mc, args_dict = load_matrix_data(to_use, A_name, B_name)
print(f"{A_name} - {B_name}, {mc.num_a} - {mc.num_b}")

result = {"matrix_stats": print_args_dict(args_dict, out=False)}
region_sizes = [mc.num_a, mc.num_b]
print(result)

In [ ]:
stats = mc.compute_stats()
print(stats)

mc_dist = stats["out_connections_dist"]
plt.plot(list(mc_dist.keys()), list(mc_dist.values()))
print(get_dist_mean(mc_dist))

In [ ]:
from skm_pyutils.table import list_to_df
new_dist = new_dist_upper_bound(mc_dist, 400)
plt.plot(list(new_dist.keys()), list(new_dist.values()),)
l = [[k, v] for k, v in new_dist.items()]
df = list_to_df(l, ["Outgoing connections", "Probability"])
print(df)

In [ ]:
# Place a probe into the brain
cylinder = place_probes_at_com(
    [A_name, B_name],
    hemisphere=hemisphere,
    colors=[myterial.blue_dark, myterial.pink_darker, myterial.deep_purple_darker],
    style="cartoon",
    interactive=False,
    probe_kwargs=dict(top_scale=0.95, angles_top=[0, 0, 3], angles_bottom=[0, 0, -2])
)

In [ ]:
if vis_full:
    o_name = f"mc_mat_vis_{A_name}_to_{B_name}.pdf"
    print("Plotting full matrix vis")
    matrix_vis(mc.ab, mc.ba, mc.aa, mc.bb, 150, name=o_name)


In [ ]:
import time


brain_region_meshes = get_brain_region_meshes(
    [A_name, B_name], None, hemisphere=hemisphere
)

t = time.perf_counter()
print("Placing cells in device")
region_pts = []
for region_mesh, region_size in zip(brain_region_meshes, region_sizes):
    pts = get_n_random_points_in_region(region_mesh, region_size, sort_=True)
    meshes = [cylinder]
    pts_idxs = np.sort(get_idx_of_points_in_meshes(pts, meshes))
    pts = pts[pts_idxs]
    region_pts.append((pts, pts_idxs))
t2 = time.perf_counter() - t
print(f"Finished cells creation in {t2:.2f}s")


In [ ]:
a_indices = region_pts[0][1]
b_indices = region_pts[1][1]

t = time.perf_counter()
print("Visualsing matrix")
mc_sub = mc.subsample(a_indices, b_indices)
o_name = f"{A_name}_to_{B_name}_connection_matrix_subbed.pdf"
matrix_vis(mc_sub.ab, mc_sub.ba, mc_sub.aa, mc_sub.bb, 40, o_name)
plt.show()
t2 = time.perf_counter() - t
print(f"Finished matrix vis in {t2:.2f}s")

In [ ]:
probe_stats = mc.compute_probe_stats(
    a_indices,
    b_indices,
)

dist = probe_stats["stats"]["out_connections_dist"]
print(100 * get_dist_mean(dist) / probe_stats["stats"]["num_end"], get_dist_var(dist))
print(100 * probe_stats["stats"]["num_senders"] / probe_stats["stats"]["num_start"])
print(probe_stats["stats"]["num_start"], probe_stats["stats"]["num_end"])

plt.plot(list(dist.keys()), list(dist.values()))

In [ ]:
res = mpf_probe_connectome(
    mc,
    [20, 20],
    a_indices,
    b_indices,
    1,
    args_dict,
    10,
    0.01,
    True,
    False
)
print(res)